# Deploying the model to a SageMaker endpoint

In [ ]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

In [ ]:
%cd model
!tar zcvf model.tar.gz *

In [ ]:
s3_location=f"s3://{sagemaker_session_bucket}/haystack-demo/model.tar.gz"
!aws s3 cp model.tar.gz $s3_location

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel

huggingface_model = HuggingFaceModel(
   model_data=s3_location,
   role=role,
   transformers_version="4.17",
   pytorch_version="1.10",
   py_version='py38',
)

In [ ]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    endpoint_name="haystack-demo-endpoint",
)

## Testing the endpoint

In [ ]:
data = {
  "inputs": "Who killed Tywin Lannister?", "parameters": {"Retriever": {"top_k": 10}, "Reader": {"top_k": 3}}
}
res = predictor.predict(data=data)

In [ ]:
import json
result = json.loads(res['response'])

In [ ]:
for r in result['answers']:
    print(f"Answer: {r['answer']}\nConfidence: {r['score']*100:.1f}%\nContext: {r['context']}\nDocument: {r['meta']['name']}\n")